# Palm Oil Futures: Part B Setup

I need to first get the "all to left" dataframe. Then, I'll group by Month and take First and Last values. 

For each month, I'll calculate its gap against the previous month's end. This will then be turned into a cumulative gap by adding them up, which can be joined to the main dataframe.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
palm_futures = pd.read_csv("C:/Users/jonah/Documents/A_MFRE_work_from_home/501/futures_assignment/shift_left.csv")

# palm_futures = pd.read_csv("C:/Users/jizatt/Documents/Courses/FRE_501/futures_contracts/shift_left.csv")

In [4]:
palm_futures.head()

,Time,cuf19
0,2016-09-22,621.50
1,2016-09-23,621.00
2,2016-09-26,614.25
3,2016-09-27,609.50
4,2016-09-28,600.00


In [5]:
palm_futures = palm_futures[palm_futures["Time"] >= "2019-01-01"]

palm_futures.set_index("Time", inplace = True)


In [6]:
# Convert the index to datetime format
palm_futures.index = pd.to_datetime(palm_futures.index)

# Check if the index is ordered in ascending "Time" order
is_ordered = palm_futures.index.is_monotonic_increasing

if is_ordered:
    print("DataFrame is ordered in ascending 'Time' order.")
else:
    print("DataFrame is not ordered in ascending 'Time' order.")

DataFrame is ordered in ascending 'Time' order.


In [7]:
palm_futures

,cuf19
Time,
2019-01-02,523.50
2019-01-03,519.25
2019-01-04,524.50
2019-01-07,524.75
2019-01-08,528.75
...,...
2023-08-15,825.25
2023-08-16,828.50
2023-08-17,835.75


In [8]:
palm_futures.index

DatetimeIndex(['2019-01-02', '2019-01-03', '2019-01-04', '2019-01-07',
               '2019-01-08', '2019-01-09', '2019-01-10', '2019-01-11',
               '2019-01-14', '2019-01-15',
               ...
               '2023-08-08', '2023-08-09', '2023-08-10', '2023-08-11',
               '2023-08-14', '2023-08-15', '2023-08-16', '2023-08-17',
               '2023-08-18', '2023-08-21'],
              dtype='datetime64[ns]', name='Time', length=1167, freq=None)

In [9]:
# turn to datetime
palm_futures.index = pd.to_datetime(palm_futures.index)

# create Month column
palm_futures['Month'] = palm_futures.index.strftime('%Y-%m')


In [10]:
month_lasts = palm_futures.groupby("Month").last()

In [11]:
month_firsts  = palm_futures.groupby("Month").first()

In [12]:
month_lasts = palm_futures.groupby("Month").last()

month_firsts  = palm_futures.groupby("Month").first()

month_profits = month_lasts["cuf19"] - month_firsts["cuf19"]

month_profits.head()

Month
2019-01    15.25
2019-02   -11.75
2019-03   -15.25
2019-04     8.00
2019-05   -18.50
Name: cuf19, dtype: float64

In [13]:
month_profits.sum()

383.25

Quick Peek at roll yield myth supposed profit: 

In [14]:
month_lasts["cuf19"].tail(1)

Month
2023-08    836.0
Name: cuf19, dtype: float64

In [15]:
month_firsts["cuf19"].head(1)

Month
2019-01    523.5
Name: cuf19, dtype: float64

In [16]:
836.0 - 523.5

312.5

# Calculating Synthetic Curve

The prices we observe for futures are not ac true representation of profits and gains from holding the asset in the long-term, due to the Roll Yield, which represents storage costs (when a positive cost) or a convenience premium for deferring commodity use (when a negative cost, or profit). We want to calculate the Synthetic Curve, which represents the actual path of profits, with gains or losses over time. 


`Price = Base Price + Within-Month Profit + Between-Month Gaps`

Both of these are cumulative over time. So there are actually two valid approaches to get the Synthetic Curve:

1. `Synthetic Curve` = `Price - Between-Month Gaps`

In this case, we would take each Month(i) end value and the Month(i+1) start value, and subtract the former from the latter to get their gap. We then cumulatively sum all month gaps and join these onto the main prices dataframe, and subtract the cumulative gap value from the real price curve. 

Since we've removed the `Between-Month Gaps`, we've got the part of the overall `Price` that's equal to the `Base Price` (which doesn't particularly matter for future monthly changes) and the `Within-Month Profit`, which we are actually interested in 


2. `Synthetic Curve` = `Base Price + Within-Month Profit`

There's another way we can calculate the Synthetic Curve, which could be easier computationally. 


`Price = Base Price + Within-Month Profit + Between-Month Gaps`

`Synthetic Curve = Price - Between-Month Gaps`

What if we substitute `Price` into `Synthetic Curve`?

`Synthetic Curve = Base Price + Within-Month Profit + Between-Month Gaps - Between-Month Gaps`

`Synthetic Curve = Base Price + Within-Month Profit`

So, if we take the base price at the start of the observation period (2019), and then add just the `Within-Month Profit` values, then we'd also get the Synthetic Curve.



Brainwave:

One problem with Approach 2 is that it's easy to calculate the End-Start values for each month, and even join these to a dataframe and add them to base price, but getting the daily shifts (without the between-month shifts) is very difficult.

In [ ]:

month_synth_prof = month_firsts.copy()

month_synth_prof["cum_profit"] = month_profits.cumsum()

month_synth_prof["synthetic_curve"] = month_synth_prof["cum_profit"] + 523.5

month_synth_prof.head()

,cuf19,month,cum_profit,synthetic_curve
Month,,,,
2019-01,523.50,2019-01-02,15.25,538.75
2019-02,562.75,2019-02-01,3.50,527.00
2019-03,537.50,2019-03-01,-11.75,511.75
2019-04,518.50,2019-04-01,-3.75,519.75
2019-05,507.00,2019-05-01,-22.25,501.25


In [ ]:
month_lasts.head()

,cuf19,month
Month,,
2019-01,538.75,2019-01-31
2019-02,551.00,2019-02-28
2019-03,522.25,2019-03-29
2019-04,526.50,2019-04-30
2019-05,488.50,2019-05-31


In [ ]:
month_firsts

,cuf19,month
Month,,
2019-01,523.50,2019-01-02
2019-02,562.75,2019-02-01
2019-03,537.50,2019-03-01
2019-04,518.50,2019-04-01
2019-05,507.00,2019-05-01
2019-06,484.75,2019-06-03
2019-07,474.00,2019-07-01
2019-08,498.25,2019-08-01
2019-09,516.25,2019-09-03
